# Abstract

Make the test comparison associations

# Environment

In [1]:
# Setup debugging log
import logging
logger = logging.getLogger('jwst.associations')
handler = logging.StreamHandler()
logger.addHandler(handler)
handler.setLevel(logging.DEBUG)
logger.setLevel(logging.DEBUG)

In [2]:
import pdb
from tempfile import TemporaryDirectory

In [3]:
import numpy as np

In [4]:
from jwst.associations import (
    AssociationRegistry,
    generate
)
from jwst.associations.main import Main

In [5]:
from jwst.associations.tests.helpers import (
    combine_pools,
    registry_level3_only,
    registry_level2_only,
    t_path,
)

# Library

# Main

## Level 2 Associations

In [6]:
save_args = [
    '-r',
    '../../lib/rules_level2b.py',
    '--ignore-default',
]

### pool_009_spec_miri_lv2bkg

In [7]:
pool_009_spec_miri_lv2bkg = combine_pools('../data/pool_009_spec_miri_lv2bkg.csv')

In [8]:
with TemporaryDirectory() as tmp_path:
    args = save_args + ['-p', tmp_path]
    results = Main(
        args,
        pool=pool_009_spec_miri_lv2bkg
    )
    %ls $tmp_path
    %mv $tmp_path/jw99009-o001_spec2_001_asn.json ../data/pool_009_spec2_001_asn.json
    %mv $tmp_path/jw99009-o003_image2_001_asn.json ../data/pool_009_image2_001_asn.json

201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_lev

jw99009-o001_spec2_001_asn.json   jw99009-o003_image2_001_asn.json


### pool_010_spec_nirspec_lv2bkg

In [9]:
pool_010_spec_nirspec_lv2bkg = combine_pools('../data/pool_010_spec_nirspec_lv2bkg.csv')

In [10]:
with TemporaryDirectory() as tmp_path:
    args = save_args + ['-p', tmp_path]
    results = Main(
        args,
        pool=pool_010_spec_nirspec_lv2bkg
    )
    %ls $tmp_path
    %mv $tmp_path/jw99009-c1000_spec2_001_asn.json ../data/pool_010_spec2_001_asn.json
    %mv $tmp_path/jw99009-o003_image2_001_asn.json ../data/pool_010_image2_001_asn.json

201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_lev

jw99009-c1000_spec2_001_asn.json  jw99009-o003_image2_001_asn.json


### pool_011_spec_miri_lv2bkg_lrs

In [11]:
pool_011_spec_miri_lv2bkg_lrs = combine_pools('../data/pool_011_spec_miri_lv2bkg_lrs.csv')

In [12]:
with TemporaryDirectory() as tmp_path:
    args = save_args + ['-p', tmp_path]
    results = Main(
        args,
        pool=pool_011_spec_miri_lv2bkg_lrs
    )
    %ls $tmp_path
    %mv $tmp_path/jw99009-o001_spec2_001_asn.json ../data/pool_011_spec2_001_asn.json

201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_lev

jw99009-o001_spec2_001_asn.json


### pool_014_ami_niriss

In [36]:
save_args = []

In [37]:
pool_014_ami_niriss = combine_pools('../data/pool_014_ami_niriss.csv')

In [38]:
with TemporaryDirectory() as tmp_path:
    args = save_args + ['-p', tmp_path]
    results = Main(
        args,
        pool=pool_014_ami_niriss
    )
    %ls $tmp_path

201709201733:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201733:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201733:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201733:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201733:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201733:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201733:WARNING:jwst.associations.lib.rules_lev

201709201733:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201733:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201733:WARNING:jwst.associations.lib.rules_level3_base.rename_to_level2b:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2b.
201709201733:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201733:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201733:WARNING:jwst.associations.lib.rules_level3_base.rename_to_level2b:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2b.
201709201733:WARNING:jwst.associations.lib.rules_lev

jw10005-a3001_ami3_001_asn.json    jw10005-c1017_ami3_001_asn.json
jw10005-a3001_ami3_002_asn.json    jw10005-c1020_ami3_001_asn.json
jw10005-a3001_ami3_003_asn.json    jw10005-c1021_ami3_001_asn.json
jw10005-a3001_ami3_004_asn.json    jw10005-c1022_ami3_001_asn.json
jw10005-a3001_ami3_005_asn.json    jw10005-c1023_ami3_001_asn.json
jw10005-a3001_ami3_006_asn.json    jw10005-c1024_ami3_001_asn.json
jw10005-c1000_ami3_001_asn.json    jw10005-c1025_ami3_001_asn.json
jw10005-c1000_image2_001_asn.json  jw10005-c1026_ami3_001_asn.json
jw10005-c1001_ami3_001_asn.json    jw10005-c1027_ami3_001_asn.json
jw10005-c1002_ami3_001_asn.json    jw10005-o005_ami3_001_asn.json
jw10005-c1003_ami3_001_asn.json    jw10005-o006_ami3_001_asn.json
jw10005-c1004_ami3_001_asn.json    jw10005-o007_ami3_001_asn.json
jw10005-c1005_ami3_001_asn.json    jw10005-o008_ami3_001_asn.json
jw10005-c1006_ami3_001_asn.json    jw10005-o009_ami3_001_asn.json
jw10005-c1007_ami3_001_asn.json    jw10005-o010_ami3_001_asn.json
j

### pool_015_spec_nirspec_lv2bkg_reversed

In [13]:
pool_015_spec_nirspec_lv2bkg_reversed = combine_pools('../data/pool_015_spec_nirspec_lv2bkg_reversed.csv')

In [14]:
with TemporaryDirectory() as tmp_path:
    args = save_args + ['-p', tmp_path]
    results = Main(
        args,
        pool=pool_015_spec_nirspec_lv2bkg_reversed
    )
    %ls $tmp_path
    %mv $tmp_path/jw99009-o001_image2_001_asn.json ../data/pool_015_image2_001_asn.json
    %mv $tmp_path/jw99009-o001_spec2_001_asn.json ../data/pool_015_spec2_001_asn.json

201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201712:WARNING:jwst.associations.lib.rules_lev

jw99009-o001_image2_001_asn.json  jw99009-o001_spec2_001_asn.json


### pool_016_spec_nirspec_lv2bkg_double

In [15]:
pool_016_spec_nirspec_lv2bkg_double = combine_pools('../data/pool_016_spec_nirspec_lv2bkg_double.csv')

In [16]:
with TemporaryDirectory() as tmp_path:
    args = save_args + ['-p', tmp_path]
    results = Main(
        args,
        pool=pool_016_spec_nirspec_lv2bkg_double
    )
    %ls $tmp_path
    %mv $tmp_path/jw99009-o003_image2_001_asn.json ../data/pool_016_image2_001_asn.json
    %mv $tmp_path/jw99009-c1000_spec2_001_asn.json ../data/pool_016_spec2_001_asn.json

201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_lev

jw99009-c1000_spec2_001_asn.json  jw99009-o003_image2_001_asn.json


### pool_017_spec_nirspec_lv2imprint

In [17]:
pool_017_spec_nirspec_lv2imprint = combine_pools('../data/pool_017_spec_nirspec_lv2imprint.csv')

In [18]:
with TemporaryDirectory() as tmp_path:
    args = save_args + ['-p', tmp_path]
    results = Main(
        args,
        pool=pool_017_spec_nirspec_lv2imprint
    )
    %ls $tmp_path
    %mv $tmp_path/jw99009-o003_image2_001_asn.json ../data/pool_017_image2_001_asn.json
    %mv $tmp_path/jw99009-a3001_spec2_001_asn.json ../data/pool_017_spec2_001_asn.json

201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_lev

jw99009-a3001_spec2_001_asn.json  jw99009-o003_image2_001_asn.json


### pool_018_all_exptypes

In [19]:
save_args = [
    '-r',
    '../../lib/rules_level2b.py',
    '--ignore-default',
]

In [20]:
pool_018_all_exptypes = combine_pools('../data/pool_018_all_exptypes.csv')

In [21]:
with TemporaryDirectory() as tmp_path:
    args = save_args + ['-p', tmp_path]
    results = Main(
        args,
        pool=pool_018_all_exptypes
    )
    %mv $tmp_path/jw99009-a3001_spec2_001_asn.json ../data/pool_018_spec2_001_asn.json
    %mv $tmp_path/jw99009-o001_image2_001_asn.json ../data/pool_018_image2_001_asn.json

201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_lev

201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_lev

### pool_019_niriss_wfss

In [22]:
save_args = []

In [23]:
pool_019_niriss_wfss = combine_pools('../data/pool_019_niriss_wfss.csv')

In [24]:
with TemporaryDirectory() as tmp_path:
    args = save_args + ['-p', tmp_path]
    results = Main(
        args,
        pool=pool_019_niriss_wfss
    )
    %mv $tmp_path/jw99009-o001_spec2_001_asn.json ../data/pool_019_spec2_001_asn.json
    %mv $tmp_path/jw99009-o001_image2_001_asn.json ../data/pool_019_image2_001_asn.json
    #%mv $tmp_path/jw99009-o001_spec3_001_asn.json ../data/pool_019_spec3_001_asn.json
    %mv $tmp_path/jw99009-o001_image3_001_asn.json ../data/pool_019_image3_001_asn.json

201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_lev

201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709201713:WARNING:jwst.associations.lib.rules_lev